# Série 1: Formalisation
## Les trois médecins

### Description
Alice, Bob et Charles sont trois patients suivis par les médecins Xavier,
Yolande et Zoé. Aujourd’hui, ces 6 personnes doivent être transférées de
l’Hôpital Guéritou vers l’Hôpital Soignetou grâce à une ambulance proposant
deux places. Les médecins ne doivent jamais se retrouver en infériorité
numérique pour prodiguer leur soins. On peut toutefois avoir des patients
dans un hôpital sans médecins. De plus l’ambulance ne fait pas de trajet à
vide.
Comment va-t-il falloir organiser les diférents trajets pour transporter
les 6 personnes d’un hôpital à l’autre, en respectant les contraintes ?

### Formalisation du problème recherche
Formalisez le problème en extrayant l’information suivante pour représenter
les différentes situations possibles. Profitez de cet exercice pour reviser les
notions du cours.

1. Donnez une représentation des états.

On peut avoir une représentation du genre S = (m, p, a) avec m le nombre de médecins, p le nombre de patients et a la position de l'ambulance

Il reste à définir l'espace d'états $\Sigma$ du système...

On peut définir l'espace d'états par: W = ({0-3}, {0-3}, {0-1})

2. Quels sont les opérateurs de transition possibles ?

3. Définissez les conditions pour lesquelles les opérateurs sont applicables.

4. En vous basant sur le _TP0_, implementez un algorithme de recherche
pour résoudre le problème en utilisant un arbre de recherche correspondant à la description que vous avez choisi.

In [1]:
print("Coucou")

Coucou
